In [1]:
# Создаем csv файл с данными
import csv

data = [
    ["title", "author", "genre", "sales", "year"],
    ["1984", "George Orwell", "Science Fiction", 5000, 1949],
    ["The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954],
    ["To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960],
    ["The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951],
    ["The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925]
]

with open('books.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [2]:
# Установим необходимую библиотеку
!pip install pyspark --> None

In [3]:
# Создаем новую Spark сессию
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Books Analysis").getOrCreate()

In [16]:
# Загрузим и прочтем данные из файла csv
df = spark.read.csv("books.csv", header=True)
df.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



In [17]:
# 2. Фильтруем данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров
df_filtered = df.filter(df["sales"] > 3000)
df_filtered.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



In [18]:
# Проверим типы данных в таблице, чтобы не появлялись ошибки при их чтении и работе с ними
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- sales: string (nullable = true)
 |-- year: string (nullable = true)



In [19]:
# В столбце "sales" имеется строковый тип данных. Исправим на целочисленный и проверим
from pyspark.sql.functions import col
df = df.withColumn('sales', col('sales').cast('int'))
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- sales: integer (nullable = true)
 |-- year: string (nullable = true)



In [20]:
# 3. Сгруппируем данные по жанру и вычислите общий объем продаж для каждого жанра
df_grouped = df.groupBy("genre").sum("sales")
df_grouped.show()

+---------------+----------+
|          genre|sum(sales)|
+---------------+----------+
|Southern Gothic|      4000|
|          Novel|      6500|
|        Fantasy|      3000|
|Science Fiction|      5000|
+---------------+----------+



In [22]:
# 4. Сортируем данные по общему объему продаж в порядке убывания
df_sorted = df_grouped.orderBy("sum(sales)", ascending=False)

In [23]:
# 5. Выводим результаты на экран
df_sorted.show()

+---------------+----------+
|          genre|sum(sales)|
+---------------+----------+
|          Novel|      6500|
|Science Fiction|      5000|
|Southern Gothic|      4000|
|        Fantasy|      3000|
+---------------+----------+

